In [ ]:
class Model:
    mixed_domain = [{'name': 'n_neurons', 'type': 'discrete', 'domain': tuple(range(20,160,20))},
          {'name': 'learning_rate', 'type': 'discrete', 'domain': (0.001,0.002,0.003,0.004)},
          {'name': 'num_layers', 'type': 'discrete', 'domain': (1,2,3,4)},
          {'name': 'rnn_type', 'type': 'discrete', 'domain': (0,1,2)},
          {'name': 'learning_period', 'type': 'discrete', 'domain': tuple(range(10,110,10))},
          {'name': 'prediction_period', 'type': 'discrete', 'domain': tuple(range(10,25,5))},
          {'name': 'max_repeats', 'type': 'discrete', 'domain': tuple(range(1,100,5))},
          {'name': 'gamma', 'type': 'discrete', 'domain': (0.99,0.98)},
          {'name': 'ema', 'type': 'discrete', 'domain': (10,20)}
         ]
    
    def get_answer(self, features):
        n_neurons = int(features[0])
        learning_rate = features[1]
        num_layers = int(features[2])
        rnn_type = int(features[3])
        learning_period = int(features[5])
        prediction_period = int(features[6])
        max_repeats = int(features[7])
        gamma = features[8]
        ema = features[9]
        
        # load data
        if gamma == 0.99 && ema == 10:
            data = pd.open_csv('data-prep-ema10-beta99.csv')
        elif gamma == 0.98 && ema == 10:
            data = pd.open_csv('data-prep-ema10-beta98.csv')
        elif gamma == 0.99 && ema == 20:
            data = pd.open_csv('data-prep-ema20-beta99.csv')
        elif gamma == 0.98 && ema == 20:
            data = pd.open_csv('data-prep-ema20-beta98.csv')
            
        
        
        n_inputs = 1
        n_outputs = 1
        batch_size = 1


In [57]:
# load data and group by the date.
import pandas as pd
gamma = 0.99
ema = 10
if gamma == 0.99 and ema == 10:
    data = pd.read_csv('data-prep-ema10-beta99.csv', parse_dates=["timestamp"])
elif gamma == 0.98 and ema == 10:
    data = pd.read_csv('data-prep-ema10-beta98.csv', parse_dates=["timestamp"])
elif gamma == 0.99 and ema == 20:
    data = pd.read_csv('data-prep-ema20-beta99.csv', parse_dates=["timestamp"])
elif gamma == 0.98 and ema == 20:
    data = pd.read_csv('data-prep-ema20-beta98.csv', parse_dates=["timestamp"])

#data.groupby(pd.to_datetime(data['timestamp']).day)

def func(ts):
    return ts.date()

groups = data.set_index('timestamp').groupby(func)
for index, df in groups:
    print(index)
    print(len(df))


2019-01-28
504
2019-01-29
504
2019-01-30
504
2019-01-31
504
2019-02-01
504
2019-02-04
504
2019-02-05
504
2019-02-06
504
2019-02-07
504
2019-02-08
504
2019-02-11
504
2019-02-12
504
2019-02-13
504
2019-02-14
504
2019-02-15
504
2019-02-18
504
2019-02-19
504
2019-02-20
504
2019-02-21
504
2019-02-22
504
2019-02-25
504
2019-02-26
504
2019-02-27
504
2019-02-28
504
2019-03-01
504
2019-03-04
504
2019-03-05
504
2019-03-06
504
2019-03-07
504
2019-03-08
504
2019-03-11
504
2019-03-12
504
2019-03-13
504
2019-03-14
504
2019-03-15
504
2019-03-18
504
2019-03-19
504
2019-03-20
504
2019-03-21
504
2019-03-22
504
2019-03-25
504
2019-03-26
504
2019-03-27
504
2019-03-28
504
2019-03-29
504
2019-04-01
504
2019-04-02
504
2019-04-03
504
2019-04-04
504
2019-04-05
504
2019-04-08
504
2019-04-09
504
2019-04-10
504
